In [6]:
import pandas as pd
import numpy as np

In [7]:
def cal_entropy(y):
    """信息熵计算
    
    参数
    -------
    y:类别编号 类型：narray， shape:{n_samples}
    
    返回
    ----
    e:信息熵  类型：float
    """
    count = np.array(pd.Series(y).value_counts())
    p = count/count.sum()
    return -np.sum(np.log2(p)*p)

def choose_features_ID3(X, y):
    """选择特征（单个）
    
    参数
    ------
    X：特征，类型：ndarray，shape：{n_samples, n_features}
    
    y: 类别编号 类型：narray， shape:{n_samples}
    
    返回
    -----
    min_fea_index：选出的特征，类型：integer
    
    entropy：信息增益，类型：float
    """
    n_samples, n_features = X.shape
    
    fea_index = 0
    max_entropy = 0
    pre_y_entropy = cal_entropy(y)
    for i in range(n_features):
        entropy_sum = 0
        row_value = X[:,i]
        for value in set(row_value):
            bools = row_value==value
            entropy_sum += np.sum(bools)/n_samples * cal_entropy(y[bools])
        entropy = pre_y_entropy-entropy_sum
        if entropy>max_entropy:
            max_entropy = entropy
            fea_index = i
    return fea_index,entropy

def tree_ID3(X, y, X_name):
    """构建决策树，采用ID3，无剪枝操作
    
    参数
    ------
    X：特征，类型：ndarray，shape：{n_samples, n_features}
    
    y: 类别编号，类型：ndarray，shape:{n_samples}
    
    X_name: 特征名，类型：ndarray，shape:{n_samples}
    """
    if not len(X):return 
    if cal_entropy(y)==0:return y[0]
    
    n_samples, n_features = X.shape
    index = choose_features_ID3(X, y)[0]
    dic = {X_name[index]:{}}
    remove_fea = X[:, index]
    for fea in set(remove_fea):
        row_bool = remove_fea==fea  # 留下的行索引
        col_bool = np.arange(n_features)!=index   # 留下的列索引
        dic[X_name[index]][fea] = tree_ID3(X[row_bool][:,col_bool], y[row_bool], X_name[col_bool])
    return dic

### 测试 

In [8]:
dataSet = np.array([
        # 1
        ['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '好瓜'],
        # 2
        ['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '好瓜'],
        # 3
        ['乌黑', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '好瓜'],
        # 4
        ['青绿', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '好瓜'],
        # 5
        ['浅白', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '好瓜'],
        # 6
        ['青绿', '稍蜷', '浊响', '清晰', '稍凹', '软粘', '好瓜'],
        # 7
        ['乌黑', '稍蜷', '浊响', '稍糊', '稍凹', '软粘', '好瓜'],
        # 8
        ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '硬滑', '好瓜'],

        # ----------------------------------------------------
        # 9
        ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '硬滑', '坏瓜'],
        # 10
        ['青绿', '硬挺', '清脆', '清晰', '平坦', '软粘', '坏瓜'],
        # 11
        ['浅白', '硬挺', '清脆', '模糊', '平坦', '硬滑', '坏瓜'],
        # 12
        ['浅白', '蜷缩', '浊响', '模糊', '平坦', '软粘', '坏瓜'],
        # 13
        ['青绿', '稍蜷', '浊响', '稍糊', '凹陷', '硬滑', '坏瓜'],
        # 14
        ['浅白', '稍蜷', '沉闷', '稍糊', '凹陷', '硬滑', '坏瓜'],
        # 15
        ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '软粘', '坏瓜'],
        # 16
        ['浅白', '蜷缩', '浊响', '模糊', '平坦', '硬滑', '坏瓜'],
        # 17
        ['青绿', '蜷缩', '沉闷', '稍糊', '稍凹', '硬滑', '坏瓜']
    ])
X = dataSet[:,:-1]
y = dataSet[:,-1]
X_name = np.array(['色泽','根蒂','敲声','纹理','脐部','触感'])
tree_ID3(X, y, X_name)

{'纹理': {'稍糊': {'触感': {'硬滑': '坏瓜', '软粘': '好瓜'}},
  '清晰': {'根蒂': {'硬挺': '坏瓜',
    '稍蜷': {'色泽': {'青绿': '好瓜', '乌黑': {'触感': {'软粘': '坏瓜', '硬滑': '好瓜'}}}},
    '蜷缩': '好瓜'}},
  '模糊': '坏瓜'}}

### 封装成类

In [9]:
class Tree_ID3:
    def __init__(self):
        pass
        
    def cal_entropy(self, y):
        """信息熵计算

        参数
        -------
        y:类别, 类型:narray, shape:{n_samples}

        返回
        ----
        e:信息熵, 类型:float
        """
        count = np.array(pd.Series(y).value_counts())
        # 每个类别的概率
        p = count/count.sum()
        # 信息熵
        return -np.sum(np.log2(p)*p)

    def choose_features_ID3(self, X, y):
        """选择特征（单个）

        参数
        ------
        X:特征, 类型:ndarray, shape:{n_samples, n_features}

        y:类别, 类型:ndarray, shape:{n_samples}

        返回
        -----
        fea_index:选出的特征, 类型:integer

        entropy:信息增益, 类型:float
        """
        n_samples, n_features = X.shape

        # 最优特征的索引
        fea_index = 0
        # 最大的信息增益
        max_entropy = 0
        # 分类前标签y的信息熵
        pre_y_entropy = self.cal_entropy(y)
        
        for i in range(n_features):
            # 初始化分类后的加权信息熵
            entropy_sum = 0
            row_value = X[:,i]
            for value in set(row_value):
                # 选中的样本索引
                bools = row_value==value
                entropy_sum += np.sum(bools)/n_samples * self.cal_entropy(y[bools])
            # 当前信息增益
            entropy = pre_y_entropy-entropy_sum
            if entropy>max_entropy:
                max_entropy = entropy
                fea_index = i
        return fea_index,entropy

    def tree_ID3(self, X, y, X_name):
        """构建决策树

        参数
        ------
        X:特征, 类型:ndarray, shape:{n_samples, n_features}

        y:类别编号, 类型:ndarray, shape:{n_samples}

        X_name:特征名, 类型:ndarray, shape:{n_samples}
        
        返回
        -----
        dic:决策树, 类型:dict
        """
        if not len(X):return 
        # 只剩一类，返回
        if self.cal_entropy(y)==0:return y[0]

        n_samples, n_features = X.shape
        index = self.choose_features_ID3(X, y)[0]
        # 决策树构建
        dic = {X_name[index]:{}}
        remove_fea = X[:, index]
        for fea in set(remove_fea):
            # 剩下的行索引
            row_bool = remove_fea==fea  
            # 剩下的列索引
            col_bool = np.arange(n_features)!=index   
            # 递归
            dic[X_name[index]][fea] = self.tree_ID3(X[row_bool][:,col_bool], y[row_bool], X_name[col_bool])
        return dic 
    
    def check(self, tree, X, X_name):
        """预测
        """
        if not len(tree) or not len(X):return
        cur_fea_name = list(tree.keys())[0]
        cur_fea_index = np.where(X_name==cur_fea_name)[0][0]
        if X[cur_fea_index] not in tree[cur_fea_name].keys():return
        if tree[cur_fea_name][X[cur_fea_index]] in self.y_name:
            return tree[cur_fea_name][X[cur_fea_index]]
        else:
            bools = np.arange(len(X))!=cur_fea_index
            return self.check(tree[cur_fea_name][X[cur_fea_index]], X[bools], X_name[bools])
    
    def fit(self, X, y, X_name):
        self.X_name = X_name
        self.y_name = list(set(y))
        self.tree = self.tree_ID3(X, y, X_name)
        
    def predict(self, X):
        res = []
        for i in range(len(X)):
            res.append(self.check(self.tree, X[i], self.X_name))
        return np.array(res)

### 测试

In [10]:
clf = Tree_ID3()
clf.fit(X, y, X_name)
predict_y = clf.predict(X)
sum(predict_y==y)==len(y)

True